In [15]:
%load_ext autoreload
%autoreload 2

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


In [16]:
import json
import os
import time
import numpy as np
import pandas as pd
from datetime import datetime
from here_location_services import LS
from here_location_services.config.routing_config import ROUTING_RETURN
from here_los_utils import *

In [17]:
LS_API_KEY = os.environ.get("HERE_API_KEY")  # Get API KEY from environment.
ls = LS(api_key=LS_API_KEY)

### Load trips data

In [18]:
trips = pd.read_csv("../data/processed/trip_od_full.csv", low_memory=False)
trips.shape

(36287, 111)

In [19]:
# Use relavent columns
cols = ['TRIPID', 'Mode', 'LINKMODE', 'TRIPPURP', 'DESTPURP1', 'TRIPTIME', 'TRAVTIME',
        'TRAVDOW', 'TRAVDATE', 'TRAVMONTH', 'TRAVYEAR', 'STARTHOUR', 'STARTIME', 
        'HomeSA1', 'CUMDIST','ORIGSA1', 'DESTSA1', 'ORIG_X', 'ORIG_Y', 'DEST_X', 'DEST_Y']
trips = trips[cols]
trips.shape

(36287, 21)

### Get trips departure time

In [20]:
# Get detailed start time of trips
trips['STARTMIN'] = trips['STARTIME'] % 60
trips['START_DETAIL_TIME'] = (trips['TRAVDATE'].astype(str) + " " + trips['STARTHOUR'].astype(str) 
                              + ":" + trips['STARTMIN'].astype(str))

In [21]:
trips.head().T

,0,1,2,3,4
TRIPID,Y12H0000107P01T01,Y12H0000107P01T02,Y12H0000107P01T03,Y12H0000107P01T04,Y12H0000107P02T01
Mode,Car,Car,Walk,Walk,Car
LINKMODE,Vehicle Driver,Vehicle Driver,Walking,Walking,Vehicle Passenger
TRIPPURP,Social,Social,Recreational,Recreational,Social
DESTPURP1,Social,At or Go Home,Recreational,At or Go Home,Social
TRIPTIME,20,10,30,30,20
TRAVTIME,20,10,30,30,20
TRAVDOW,Sunday,Sunday,Sunday,Sunday,Sunday
TRAVDATE,27/05/2012,27/05/2012,27/05/2012,27/05/2012,27/05/2012
TRAVMONTH,May,May,May,May,May


### Collect Level of Service from HERE APIs

In [22]:
def get_datetime(str_time):
    return datetime.strptime(str_time,'%d/%m/%Y %H:%M')

# Test
print(get_datetime("27/05/2012 23:30"))

def get_trip_los(trip):
    time.sleep(0.3)
    start_time = get_datetime(trip['START_DETAIL_TIME'])
    orig = [trip.ORIG_Y, trip.ORIG_X]
    dest = [trip.DEST_Y, trip.DEST_X]
    (trip['CAR_TIME'], trip['CAR_LEN'], trip['CYCLE_TIME'], trip['CYCLE_LEN'], 
     trip['WALK_TIME'], trip['WALK_LEN']) = collect_here_los(ls, orig, dest, start_time)
    return trip

2012-05-27 23:30:00


In [23]:
trips.Mode.value_counts()

Walk     17115
Car      15764
Train     1704
Cycle     1704
Name: Mode, dtype: int64

In [24]:
# Divide data into walk_trips and not_walk_trips for smaller APIs number of queries
car_trips = trips[trips.Mode.isin(['Car'])]
walk_trips = trips[trips.Mode.isin(['Walk'])]
train_trips = trips[trips.Mode.isin(['Train'])]
cycle_trips = trips[trips.Mode.isin(['Cycle'])]
assert len(trips) == len(car_trips) + len(walk_trips) + len(train_trips) + len(cycle_trips)
walk_trips.shape, car_trips.shape, train_trips.shape, cycle_trips.shape, trips.shape

((17115, 23), (15764, 23), (1704, 23), (1704, 23), (36287, 23))

In [36]:
# Use batch processing as limitation of HERE service
# ApiError: 429, Too Many Requests, {"error":"Too Many Requests","error_description":"Rate limit for this service has been reached"}

def process_trips_los(trips_df, batch_size, save_path, df_name):
    """
    Process trips and get los from HERE APIs
    Parameters
        trips_df: trips dataframe
        batch_size: process in batch due to quota limitation of HERE APIs
        save_path: where to save collected batch files
    """
    batch_idxs = np.arange(len(trips_df)) // batch_size
    for gr, df in trips_df.groupby(batch_idxs):        
        trip_los = df.apply(lambda x: get_trip_los(x), axis=1)
        trip_los.to_csv(f"{save_path}/{df_name}_{gr}.csv")
        print(f"Finished batch {gr+1}", df.shape, trip_los.shape)
        time.sleep(5)
        
        
def join_trips_los(trips_len, batch_size, save_path, df_name):
    batch_idxs = np.arange(trips_len // batch_size + 1)
    trips = []
    for idx in batch_idxs:
        trip_los = pd.read_csv(f"{save_path}/{df_name}_{idx}.csv")
        trips.append(trip_los)
    return pd.concat(trips)

In [29]:
%%time
# RUN ONCE
save_path = "../data/los_here"
batch_size = 200
process_trips_los(train_trips, batch_size, save_path, "train_trips_los")

Finished batch 1 (200, 23) (200, 29)
Finished batch 2 (200, 23) (200, 29)
Finished batch 3 (200, 23) (200, 29)
Finished batch 4 (200, 23) (200, 29)
Finished batch 5 (200, 23) (200, 29)
Finished batch 6 (200, 23) (200, 29)
Finished batch 7 (200, 23) (200, 29)
Finished batch 8 (200, 23) (200, 29)
Finished batch 9 (104, 23) (104, 29)
CPU times: user 2min 23s, sys: 3.82 s, total: 2min 27s
Wall time: 48min 7s


In [14]:
%%time
# RUN ONCE
save_path = "../data/los_here"
batch_size = 200
process_trips_los(cycle_trips, batch_size, save_path, "cycle_trips_los")

Finished batch 1 (200, 23) (200, 29)
Finished batch 2 (200, 23) (200, 29)
Finished batch 3 (200, 23) (200, 29)
Finished batch 4 (200, 23) (200, 29)
Finished batch 5 (200, 23) (200, 29)
Finished batch 6 (200, 23) (200, 29)
Finished batch 7 (200, 23) (200, 29)
Finished batch 8 (200, 23) (200, 29)
Finished batch 9 (104, 23) (104, 29)
CPU times: user 2min 36s, sys: 3.58 s, total: 2min 39s
Wall time: 47min 3s


In [26]:
%%time
# RUN ONCE
save_path = "../data/los_here"
batch_size = 200
process_trips_los(car_trips, batch_size, save_path, "car_trips_los")

Finished batch 1 (200, 23) (200, 29)
Finished batch 2 (200, 23) (200, 29)
Finished batch 3 (200, 23) (200, 29)
Finished batch 4 (200, 23) (200, 29)
Finished batch 5 (200, 23) (200, 29)
Finished batch 6 (200, 23) (200, 29)
Finished batch 7 (200, 23) (200, 29)
Finished batch 8 (200, 23) (200, 29)
Finished batch 9 (200, 23) (200, 29)
Finished batch 10 (200, 23) (200, 29)
Finished batch 11 (200, 23) (200, 29)
Finished batch 12 (200, 23) (200, 29)
Finished batch 13 (200, 23) (200, 29)
Finished batch 14 (200, 23) (200, 29)
Finished batch 15 (200, 23) (200, 29)
Finished batch 16 (200, 23) (200, 29)
Finished batch 17 (200, 23) (200, 29)
Finished batch 18 (200, 23) (200, 29)
Finished batch 19 (200, 23) (200, 29)
Finished batch 20 (200, 23) (200, 29)
Finished batch 21 (200, 23) (200, 29)
Finished batch 22 (200, 23) (200, 29)
Finished batch 23 (200, 23) (200, 29)
Finished batch 24 (200, 23) (200, 29)
Finished batch 25 (200, 23) (200, 29)
Finished batch 26 (200, 23) (200, 29)
Finished batch 27 (20

In [27]:
walk_trips.shape

(17115, 23)

In [28]:
%%time
# RUN ONCE
save_path = "../data/los_here"
batch_size = 200
process_trips_los(walk_trips, batch_size, save_path, "walk_trips_los")

Finished batch 1 (200, 23) (200, 29)
Finished batch 2 (200, 23) (200, 29)
Finished batch 3 (200, 23) (200, 29)
Finished batch 4 (200, 23) (200, 29)
Finished batch 5 (200, 23) (200, 29)
Finished batch 6 (200, 23) (200, 29)
Finished batch 7 (200, 23) (200, 29)
Finished batch 8 (200, 23) (200, 29)
Finished batch 9 (200, 23) (200, 29)
Finished batch 10 (200, 23) (200, 29)
Finished batch 11 (200, 23) (200, 29)
Finished batch 12 (200, 23) (200, 29)
Finished batch 13 (200, 23) (200, 29)
Finished batch 14 (200, 23) (200, 29)
Finished batch 15 (200, 23) (200, 29)
Finished batch 16 (200, 23) (200, 29)
Finished batch 17 (200, 23) (200, 29)
Finished batch 18 (200, 23) (200, 29)
Finished batch 19 (200, 23) (200, 29)
Finished batch 20 (200, 23) (200, 29)
Finished batch 21 (200, 23) (200, 29)
Finished batch 22 (200, 23) (200, 29)
Finished batch 23 (200, 23) (200, 29)
Finished batch 24 (200, 23) (200, 29)
Finished batch 25 (200, 23) (200, 29)
Finished batch 26 (200, 23) (200, 29)
Finished batch 27 (20

In [75]:
# THIS WILL RETURN ERROR
# # ApiError: 429, Too Many Requests, {"error":"Too Many Requests","error_description":"Rate limit for this service has been reached"}
# trip_los = trips.apply(lambda x: get_trip_los(x), axis=1)
# trip_los.shapefull_trips_los

### Join batch files together

In [39]:
save_path = "../data/los_here"
batch_size = 200
train_trips_los = join_trips_los(len(train_trips), batch_size, save_path, "train_trips_los")
car_trips_los = join_trips_los(len(car_trips), batch_size, save_path, "car_trips_los")
walk_trips_los = join_trips_los(len(walk_trips), batch_size, save_path, "walk_trips_los")
cycle_trips_los = join_trips_los(len(cycle_trips), batch_size, save_path, "cycle_trips_los")
train_trips_los.shape, car_trips_los.shape, walk_trips_los.shape, cycle_trips_los.shape

((1704, 30), (15764, 30), (17115, 30), (1704, 30))

In [69]:
full_trips_los = pd.concat([train_trips_los, car_trips_los, walk_trips_los, cycle_trips_los])
full_trips_los.shape

(36287, 30)

In [70]:
# Remove trips with CAR_TIME == 0
full_trips_los = full_trips_los[(full_trips_los.CAR_TIME != 0) & (full_trips_los.WALK_TIME != 0)]
full_trips_los.drop(columns=["Unnamed: 0"], inplace=True)
full_trips_los.shape

(32904, 29)

In [71]:
full_trips_los.Mode.value_counts()

Car      15448
Walk     14115
Train     1695
Cycle     1646
Name: Mode, dtype: int64

In [72]:
full_trips_los.head()

,TRIPID,Mode,LINKMODE,TRIPPURP,DESTPURP1,TRIPTIME,TRAVTIME,TRAVDOW,TRAVDATE,TRAVMONTH,...,DEST_X,DEST_Y,STARTMIN,START_DETAIL_TIME,CAR_TIME,CAR_LEN,CYCLE_TIME,CYCLE_LEN,WALK_TIME,WALK_LEN
0,Y12H0000303P01T01,Train,Train,Work,Work Related,30,30,Wednesday,23/05/2012,May,...,144.997645,-37.831626,10,23/05/2012 7:10,747,9336,2295,9205,8867,8733
1,Y12H0000303P01T02,Train,Train,Work,At or Go Home,40,35,Wednesday,23/05/2012,May,...,145.061520,-37.872402,0,23/05/2012 14:0,686,10328,2330,9192,8869,8782
2,Y12H0000312P01T01,Train,Train,Work,Work Related,55,47,Friday,25/05/2012,May,...,144.963048,-37.817061,0,25/05/2012 8:0,846,13118,3174,12544,12363,12250
3,Y12H0000312P01T04,Train,Train,Work,At or Go Home,65,55,Friday,25/05/2012,May,...,145.061520,-37.872402,15,25/05/2012 18:15,871,13612,87,57,12388,12250
4,Y12H0000326P03T01,Train,Train,Work,Work Related,55,48,Friday,25/05/2012,May,...,144.970239,-37.814414,10,25/05/2012 8:10,984,15354,3724,15021,14415,14272


### Merge with trips_od to get full trip info

In [73]:
trips_od = pd.read_csv("../data/processed/trip_od_full.csv", low_memory=False)
cols = ['Mode', 'LINKMODE', 'TRIPPURP', 'DESTPURP1', 'TRIPTIME', 'TRAVTIME',
        'TRAVDOW', 'TRAVDATE', 'TRAVMONTH', 'TRAVYEAR', 'STARTHOUR', 'STARTIME', 
        'HomeSA1', 'CUMDIST','ORIGSA1', 'DESTSA1', 'ORIG_X', 'ORIG_Y', 'DEST_X', 'DEST_Y']
trips_od = trips_od.drop(columns=cols)

In [74]:
full_trips_los = full_trips_los.merge(trips_od, how='left', on='TRIPID')

In [78]:
# Save trips with LoS for further analysis
full_trips_los.to_csv("../data/processed/trip_here_los.csv", index=False)

In [79]:
list(full_trips_los.columns)

['TRIPID',
 'Mode',
 'LINKMODE',
 'TRIPPURP',
 'DESTPURP1',
 'TRIPTIME',
 'TRAVTIME',
 'TRAVDOW',
 'TRAVDATE',
 'TRAVMONTH',
 'TRAVYEAR',
 'STARTHOUR',
 'STARTIME',
 'HomeSA1',
 'CUMDIST',
 'ORIGSA1',
 'DESTSA1',
 'ORIG_X',
 'ORIG_Y',
 'DEST_X',
 'DEST_Y',
 'STARTMIN',
 'START_DETAIL_TIME',
 'CAR_TIME',
 'CAR_LEN',
 'CYCLE_TIME',
 'CYCLE_LEN',
 'WALK_TIME',
 'WALK_LEN',
 'PERSID',
 'TRIPNO',
 'ARRHOUR',
 'ARRTIME',
 'WAITIME',
 'ORIGSA3',
 'DESTSA3',
 'ORIG_CENTROID',
 'DEST_CENTROID',
 'HHID',
 'PERSNO',
 'WEEK',
 'MONTHOFBIRTH',
 'YEAROFBIRTH',
 'AGE',
 'AGEGROUP',
 'SEX',
 'RELATIONSHIP',
 'CARLICENCE',
 'MBIKELICENCE',
 'OTHERLICENCE',
 'NOLICENCE',
 'FULLTIMEWORK',
 'PARTTIMEWORK',
 'CASUALWORK',
 'ANYWORK',
 'STUDYING',
 'ACTIVITIES',
 'MAINACT',
 'WORKTYPE',
 'EMPTYPE',
 'ANZSCO1',
 'ANZSIC1',
 'STARTPLACE',
 'ANYSTOPS',
 'NUMSTOPS',
 'REASONCODE',
 'MRTDOW',
 'MRTINT',
 'PERSINC',
 'WHOFILLED',
 'PROXY',
 'FILLDOW',
 'FILLLAG',
 'ADDITIONALTRAVEL',
 'NEWSTOP',
 'DISABLEDTRANS',